**DL Assignment 3**  
**Name - Sanyam Kaul**  
**Roll Number - CS23MTECH14011**

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

In [17]:
BATCH_SIZE = 64
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = CIFAR10(root='./data', train=False, transform=transform, download=True)

train_loader_custom = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader_custom = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


Q1

In [31]:
class SelfAttention(nn.Module):
    def __init__(self, in_channels, reduction_ratio=8):
        super(SelfAttention, self).__init__()
        # Query convolution layer to compute query projections
        self.query_conv = nn.Conv2d(in_channels, in_channels // reduction_ratio, kernel_size=1)
        # Key convolution layer to compute key projections
        self.key_conv = nn.Conv2d(in_channels, in_channels // reduction_ratio, kernel_size=1)
        # Value convolution layer to compute value projections
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        # Learnable parameter for scaling the attention output
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        # Retrieve dimensions of the input tensor
        batch_size, channels, width, height = x.size()

        # Compute query projections and reshape for matrix multiplication
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        # proj_query: [batch_size, width * height, channels // reduction_ratio]

        # Compute key projections and reshape for matrix multiplication
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        # proj_key: [batch_size, channels // reduction_ratio, width * height]

        # Compute energy (dot product of query and key) using batch matrix multiplication
        energy = torch.bmm(proj_query, proj_key)
        # energy: [batch_size, width * height, width * height]

        # Apply softmax along the last dimension to compute attention scores
        attention = F.softmax(energy, dim=-1)

        # Compute value projections and reshape for matrix multiplication
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        # proj_value: [batch_size, channels, width * height]

        # Compute output by multiplying attention scores with value projections
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        # out: [batch_size, channels, width * height]

        # Reshape output to match the original input dimensions
        out = out.view(batch_size, channels, width, height)
        # out: [batch_size, channels, width, height]

        # Scale the output using the gamma parameter and add a residual connection
        out = (self.gamma * out) + x
        return out


In [32]:
# Simple CNN Model with Self-Attention
class SimpleCNNWithSelfAttention(nn.Module):
    def __init__(self):
        super(SimpleCNNWithSelfAttention, self).__init__()
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(64, 10, kernel_size=3, padding=1)
        # Self-attention layers
        self.sa1 = SelfAttention(16)
        self.sa2 = SelfAttention(32)
        self.sa3 = SelfAttention(64)
        self.sa4 = SelfAttention(64)

    def forward(self, x):
        # Input dimensions: [batch_size, 3, height, width]

        # First convolutional layer with ReLU activation
        x = F.relu(self.conv1(x))
        # Output dimensions: [batch_size, 16, height, width]

        # Apply self-attention
        x = self.sa1(x)

        # Second convolutional layer with ReLU activation
        x = F.relu(self.conv2(x))
        # Output dimensions: [batch_size, 32, height, width]

        # Max pooling layer
        x = F.max_pool2d(x, 2)
        # Output dimensions: [batch_size, 32, height/2, width/2]

        # Apply self-attention
        x = self.sa2(x)

        # Third convolutional layer with ReLU activation
        x = F.relu(self.conv3(x))
        # Output dimensions: [batch_size, 64, height/2, width/2]

        # Max pooling layer
        x = F.max_pool2d(x, 2)
        # Output dimensions: [batch_size, 64, height/4, width/4]

        # Apply self-attention
        x = self.sa3(x)

        # Fourth convolutional layer with ReLU activation
        x = F.relu(self.conv4(x))
        # Output dimensions: [batch_size, 64, height/4, width/4]

        # Apply self-attention
        x = self.sa4(x)

        # Fifth convolutional layer with ReLU activation
        x = F.relu(self.conv5(x))
        # Output dimensions: [batch_size, 64, height/4, width/4]

        # Sixth convolutional layer with ReLU activation
        x = F.relu(self.conv6(x))
        # Output dimensions: [batch_size, 10, height/4, width/4]

        # Global average pooling
        x = F.avg_pool2d(x, 8)
        # Output dimensions: [batch_size, 10, 1, 1]

        # Reshape for classification
        x = x.view(-1, 10)
        # Output dimensions: [batch_size, 10]

        # Apply log softmax for classification
        x = F.log_softmax(x, dim=-1)
        return x

In [33]:
# Training Function
def train(model, train_loader, optimizer, criterion, num_epochs=60):
    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        total_batches = len(train_loader)
        for i, (inputs, labels) in enumerate(train_loader):
            # Move inputs and labels to GPU if available
            inputs, labels = inputs.cuda(), labels.cuda() if torch.cuda.is_available() else (inputs, labels)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Calculate epoch-wise average loss
        epoch_loss = running_loss / total_batches

        # Print epoch-wise average loss
        print(f"Epoch [{epoch + 1}/{num_epochs}], Average Loss: {epoch_loss:.3f}")

    # Print when training is finished
    print('Finished Training')


In [34]:
# Evaluation Function
def evaluate(model, test_loader):
    # Set the model to evaluation mode
    model.eval()

    # Initialize variables for tracking correct predictions and total samples
    correct = 0
    total = 0

    # Disable gradient calculation during evaluation
    with torch.no_grad():
        # Iterate over the test data loader
        for images, labels in test_loader:
            # Move images and labels to GPU if available
            images, labels = images.cuda(), labels.cuda() if torch.cuda.is_available() else (images, labels)

            # Perform forward pass to get predictions
            outputs = model(images)

            # Get the predicted labels
            _, predicted = torch.max(outputs.data, 1)

            # Update the total number of samples
            total += labels.size(0)

            # Update the number of correct predictions
            correct += (predicted == labels).sum().item()

    # Calculate accuracy
    accuracy = 100 * correct / total

    # Print the accuracy of the model on the test dataset
    print('Accuracy of the network on the test dataset: %d %%' % accuracy)


In [35]:
# Main function
def main():
    # Instantiate the model
    model = SimpleCNNWithSelfAttention().cuda() if torch.cuda.is_available() else SimpleCNNWithSelfAttention()

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    train(model, train_loader_custom, optimizer, criterion, num_epochs=20)

    # Evaluate the model
    evaluate(model, test_loader_custom)

In [23]:
if __name__ == "__main__":
    main()

Epoch [1/20], Average Loss: 1.822
Epoch [2/20], Average Loss: 1.366
Epoch [3/20], Average Loss: 1.189
Epoch [4/20], Average Loss: 1.073
Epoch [5/20], Average Loss: 0.975
Epoch [6/20], Average Loss: 0.744
Epoch [7/20], Average Loss: 0.609
Epoch [8/20], Average Loss: 0.550
Epoch [9/20], Average Loss: 0.500
Epoch [10/20], Average Loss: 0.457
Epoch [11/20], Average Loss: 0.413
Epoch [12/20], Average Loss: 0.381
Epoch [13/20], Average Loss: 0.349
Epoch [14/20], Average Loss: 0.317
Epoch [15/20], Average Loss: 0.291
Epoch [16/20], Average Loss: 0.267
Epoch [17/20], Average Loss: 0.243
Epoch [18/20], Average Loss: 0.217
Epoch [19/20], Average Loss: 0.204
Epoch [20/20], Average Loss: 0.186
Finished Training
Accuracy of the network on the test dataset: 79 %


Q2

In [36]:
# Customizing Constants
NUM_CLASSES_CUSTOM = 10  # Number of classes in the dataset
IMG_SIZE_CUSTOM = 64  # Input image size (assuming square images)
PATCH_SIZE_CUSTOM = 4  # Patch size used in the Vision Transformer
NUM_PATCHES_CUSTOM = (IMG_SIZE_CUSTOM // PATCH_SIZE_CUSTOM) ** 2  # Number of patches in the image
EMBED_DIM_CUSTOM = 128  # Embedding dimension for each patch
NUM_HEADS_CUSTOM = 16  # Number of attention heads in the Multi-Head Attention mechanism
NUM_ENCODER_LAYERS_CUSTOM = 12  # Number of encoder layers in the Vision Transformer
MLP_DIM_CUSTOM = 256  # Dimension of the feedforward network in the Transformer Encoder Layer
DROPOUT_CUSTOM = 0.1  # Dropout rate used in various parts of the model
BATCH_SIZE_CUSTOM = 64  # Batch size used during training
NUM_EPOCHS_CUSTOM = 100  # Number of epochs for training
LEARNING_RATE_CUSTOM = 0.001  # Learning rate used by the optimizer during training


In [37]:
# Define Vision Transformer model
class VisionTransformer(nn.Module):
    def __init__(self):
        super(VisionTransformer, self).__init__()
        # Patch embedding layer
        self.patch_embed = nn.Conv2d(in_channels=3, out_channels=EMBED_DIM_CUSTOM, kernel_size=PATCH_SIZE_CUSTOM, stride=PATCH_SIZE_CUSTOM)
        # Positional embedding parameter
        self.pos_embedding = nn.Parameter(torch.randn(1, NUM_PATCHES_CUSTOM + 1, EMBED_DIM_CUSTOM))
        # Dropout layer
        self.dropout = nn.Dropout(DROPOUT_CUSTOM)
        # List of transformer encoder layers
        self.transformer_encoder_layers = nn.ModuleList([
            CustomTransformerEncoderLayer(EMBED_DIM_CUSTOM, NUM_HEADS_CUSTOM, MLP_DIM_CUSTOM, DROPOUT_CUSTOM)
            for _ in range(NUM_ENCODER_LAYERS_CUSTOM)
        ])
        # Fully connected layer for classification
        self.fc = nn.Linear(EMBED_DIM_CUSTOM, NUM_CLASSES_CUSTOM)

    def forward(self, x):
        # Input dimensions: [batch_size, 3, height, width]

        # Extract patches and flatten them
        x = self.patch_embed(x).flatten(2).transpose(1, 2)
        # Output dimensions: [batch_size, num_patches, embed_dim]

        batch_size, num_patches, _ = x.shape

        # Add position embeddings
        x += self.pos_embedding[:, :num_patches, :]
        # Output dimensions: [batch_size, num_patches, embed_dim]

        # Transformer encoder
        for encoder_layer in self.transformer_encoder_layers:
            x = encoder_layer(x)
            # Output dimensions: [batch_size, num_patches, embed_dim]

        # Global average pooling
        x = x.mean(dim=1)
        # Output dimensions: [batch_size, embed_dim]

        # Classification
        x = self.fc(x)
        # Output dimensions: [batch_size, num_classes]

        return x

In [38]:
# Define Transformer Encoder Layer
class CustomTransformerEncoderLayer(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout):
        super(CustomTransformerEncoderLayer, self).__init__()
        # Self-attention mechanism
        self.self_attention = SelfAttention(embed_dim, num_heads, dropout)
        # First linear transformation in the feedforward network
        self.linear1 = nn.Linear(embed_dim, mlp_dim)
        # Dropout layer for the first linear transformation
        self.dropout1 = nn.Dropout(dropout)
        # Second linear transformation in the feedforward network
        self.linear2 = nn.Linear(mlp_dim, embed_dim)
        # Dropout layer for the second linear transformation
        self.dropout2 = nn.Dropout(dropout)
        # Layer normalization for the first sub-layer
        self.norm1 = nn.LayerNorm(embed_dim)
        # Layer normalization for the second sub-layer
        self.norm2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        # Input dimensions: [batch_size, num_patches, embed_dim]

        # Self-attention mechanism
        attn_output = self.self_attention(x)
        # Output dimensions: [batch_size, num_patches, embed_dim]

        # Residual connection and layer normalization for the first sub-layer
        x = x + self.dropout1(attn_output)
        x = self.norm1(x)
        # Output dimensions: [batch_size, num_patches, embed_dim]

        # Feedforward network
        x_ff = self.dropout2(F.relu(self.linear1(x)))
        # Output dimensions: [batch_size, num_patches, mlp_dim]

        x_ff = self.linear2(x_ff)
        # Output dimensions: [batch_size, num_patches, embed_dim]

        # Residual connection and layer normalization for the second sub-layer
        x = x + x_ff
        x = self.norm2(x)
        # Output dimensions: [batch_size, num_patches, embed_dim]

        return x


In [39]:
# Define Self-Attention mechanism
class SelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads, dropout):
        super(SelfAttention, self).__init__()
        # Initialize dimensions
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        # Verify embedding dimension is divisible by the number of heads
        assert self.head_dim * num_heads == embed_dim, "Embedding dimension must be divisible by the number of heads."

        # Linear transformations for query, key, and value
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)

        # Linear transformation for output
        self.fc_out = nn.Linear(embed_dim, embed_dim)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Input dimensions: [batch_size, num_patches, embed_dim]
        batch_size, num_patches, embed_dim = x.shape

        # Compute query, key, and value
        queries = self.query(x).view(batch_size, num_patches, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        keys = self.key(x).view(batch_size, num_patches, self.num_heads, self.head_dim).permute(0, 2, 1, 3)
        values = self.value(x).view(batch_size, num_patches, self.num_heads, self.head_dim).permute(0, 2, 1, 3)

        # Compute attention scores
        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attention_probs = F.softmax(attention_scores, dim=-1)

        # Apply dropout to attention probabilities
        attention_probs = self.dropout(attention_probs)

        # Compute weighted sum using attention probabilities
        weighted_sum = torch.matmul(attention_probs, values)

        # Reshape and concatenate attention heads
        weighted_sum = weighted_sum.permute(0, 2, 1, 3).contiguous().view(batch_size, num_patches, embed_dim)

        # Linear transformation
        attention_output = self.fc_out(weighted_sum)
        # Output dimensions: [batch_size, num_patches, embed_dim]

        return attention_output


In [40]:
# Initialize Custom Vision Transformer model and move it to GPU if available
custom_model = VisionTransformer().cuda() if torch.cuda.is_available() else VisionTransformer()

# Define loss function and optimizer
custom_criterion = nn.CrossEntropyLoss()
custom_optimizer = torch.optim.Adam(custom_model.parameters(), lr=LEARNING_RATE_CUSTOM)


In [29]:
# Training loop
for epoch in range(NUM_EPOCHS_CUSTOM):
    running_loss = 0.0
    custom_model.train()  # Set the model to training mode
    for i, (images, labels) in enumerate(train_loader_custom):
        images, labels = images.cuda(), labels.cuda() if torch.cuda.is_available() else (images, labels)  # Move data to GPU if available
        custom_optimizer.zero_grad()  # Zero the parameter gradients

        # Forward pass
        outputs = custom_model(images)
        # Loss computation
        loss = custom_criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        custom_optimizer.step()

        running_loss += loss.item()

    # Print average loss after each epoch
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS_CUSTOM}], Average Loss: {running_loss / len(train_loader_custom)}")

print('Finished Training')


Epoch [1/100], Average Loss: 1.9099493714244775
Epoch [2/100], Average Loss: 1.5707875031339542
Epoch [3/100], Average Loss: 1.4096964558067224
Epoch [4/100], Average Loss: 1.3236971945713853
Epoch [5/100], Average Loss: 1.2567296307867446
Epoch [6/100], Average Loss: 1.200068040653263
Epoch [7/100], Average Loss: 1.1538873073237632
Epoch [8/100], Average Loss: 1.1163295944176062
Epoch [9/100], Average Loss: 1.0857787403609136
Epoch [10/100], Average Loss: 1.0502233400826564
Epoch [11/100], Average Loss: 1.0314854657863413
Epoch [12/100], Average Loss: 1.0032276107222222
Epoch [13/100], Average Loss: 0.9789106804696496
Epoch [14/100], Average Loss: 0.955948245113768
Epoch [15/100], Average Loss: 0.9371190815020705
Epoch [16/100], Average Loss: 0.9189198440145654
Epoch [17/100], Average Loss: 0.8970288672410619
Epoch [18/100], Average Loss: 0.8853309154891602
Epoch [19/100], Average Loss: 0.8621320892935214
Epoch [20/100], Average Loss: 0.8517291859135299
Epoch [21/100], Average Loss: 0

In [30]:
# Evaluation on test dataset
custom_model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader_custom:
        images, labels = images.cuda(), labels.cuda() if torch.cuda.is_available() else (images, labels)  # Move data to GPU if available
        outputs = custom_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Compute accuracy
accuracy = 100 * correct / total

# Print final accuracy
print('Accuracy of the network on the test dataset: %d %%' % accuracy)


Accuracy of the network on the test dataset: 65 %
